In [466]:
import numpy as np
import wave
from scipy.io import wavfile
from scipy import signal
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Reshape, Input
from tensorflow.keras.layers import Convolution1D, MaxPooling1D, BatchNormalization

In [30]:
def _wav2array(nchannels, sampwidth, data):
    """data must be the string containing the bytes from the wav file."""
    num_samples, remainder = divmod(len(data), sampwidth * nchannels)
    if remainder > 0:
        raise ValueError('The length of data is not a multiple of '
                         'sampwidth * num_channels.')
    if sampwidth > 4:
        raise ValueError("sampwidth must not be greater than 4.")

    if sampwidth == 3:
        a = np.empty((num_samples, nchannels, 4), dtype=np.uint8)
        raw_bytes = np.fromstring(data, dtype=np.uint8)
        a[:, :, :sampwidth] = raw_bytes.reshape(-1, nchannels, sampwidth)
        a[:, :, sampwidth:] = (a[:, :, sampwidth - 1:sampwidth] >> 7) * 255
        result = a.view('<i4').reshape(a.shape[:-1])
    else:
        # 8 bit samples are stored as unsigned ints; others as signed ints.
        dt_char = 'u' if sampwidth == 1 else 'i'
        a = np.fromstring(data, dtype='<%s%d' % (dt_char, sampwidth))
        result = a.reshape(-1, nchannels)
    return result


def readwav(file):
    """
    Read a wav file.
    Returns the frame rate, sample width (in bytes) and a numpy array
    containing the data.
    This function does not read compressed wav files.
    """
    wav = wave.open(file)
    rate = wav.getframerate()
    nchannels = wav.getnchannels()
    sampwidth = wav.getsampwidth()
    nframes = wav.getnframes()
    data = wav.readframes(nframes)
    wav.close()
    array = _wav2array(nchannels, sampwidth, data)
    return rate, sampwidth, array

print(arr.shape)
print(arr[3000])

(45056, 2)
[-2436     0]


/Users/josephcleveland/Summer/venv/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [427]:
X_data = np.zeros(shape=(6000,45056))
print(X_data.shape)

for r in range(0, 6000):
    audio = readwav('/Users/josephcleveland/Dataset/samp' + str(r) + '.wav')[2]
    audio = audio.T[0]
    if(audio.shape != 45056):
        padded = np.zeros(shape=(45056))
        padded[:np.shape(audio)[0]] = audio
        audio = padded
    X_data[r] = audio
    


(6000, 45056)


/Users/josephcleveland/Summer/venv/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [96]:
Y_data = np.zeros(shape=(6000, 368))
mat = np.genfromtxt('/Users/josephcleveland/Dataset/mat.txt', delimiter=',')
for i in range(0, 6000):
    onehot = to_categorical(mat[i], 16)
    Y_data[i] = onehot.ravel()

In [479]:
#X_data = signal.resample(X_data, 16384)
X_train = X_data[0:5000]
X_test = X_data[5000:6000]
Y_train = Y_data[0:5000]
Y_test = Y_data[5000:6000]
X_train = np.expand_dims(X_train, axis=2)
X_train = np.expand_dims(X_train, axis=3)

#X_train_samp = np.zeros((5000, 16384, 1, 1))
#for i in range(0, 5000):
#    X_train_samp[i] = signal.resample(X_train[i], 16384)
#    print(X_train_samp[i].shape)
print(X_train.shape)

(5000, 45056, 1, 1)


In [458]:
#model = Sequential()
#model.add(BatchNormalization())
#model.add(Convolution2D(96, kernel_size=(64, 1), strides=(4, 1), padding="valid", activation='relu', data_format="channels_last", input_shape=(16384, 1, 1)))
#model.add(Convolution2D(96, kernel_size=(32, 1), strides=(4, 1), padding="valid", activation='relu'))
#model.add(Convolution2D(128, kernel_size=(16, 1), strides=(4, 1), padding="valid", activation='relu'))
#model.add(Convolution2D(257, kernel_size=(8, 1), strides=(4, 1), padding="valid", activation='relu'))
#model.add(Reshape((173, 257, 1)))
#model.add(Reshape((61, 257, 1)))
#model.add(Convolution2D(32, kernel_size=(3, 3), strides=(2, 2), padding="same", activation='relu'))
#model.add(Convolution2D(71, kernel_size=(3, 3), strides=(2, 2), padding="same", activation='relu'))
#model.add(Convolution2D(128, kernel_size=(4, 3), strides=(3, 2), padding="same", activation='relu'))
#model.add(Convolution2D(128, kernel_size=(3, 3), strides=(2, 2), padding="same", activation='relu'))
#model.add(Convolution2D(128, kernel_size=(3, 3), strides=(2, 2), padding="same", activation='relu'))
#model.add(Convolution2D(128, kernel_size=(3, 3), strides=(2, 1), padding="same", activation='relu'))
#model.add(Flatten())
#model.add(Dense(512, activation='relu'))
#model.add(Dense(368, activation='relu'))

#model.compile(loss='categorical_crossentropy',
#              optimizer='adam',
#              metrics=['accuracy'])

In [490]:
inputs = Input(shape=(45056,1,1))

# a layer instance is callable on a tensor, and returns a tensor
x = BatchNormalization()(inputs)
x = Convolution2D(96, kernel_size=(64, 1), strides=(4, 1), padding="valid", activation='relu', data_format="channels_last", input_shape=(45056, 1, 1))(x)
x = Convolution2D(96, kernel_size=(32, 1), strides=(4, 1), padding="valid", activation='relu')(x)
x = Convolution2D(128, kernel_size=(16, 1), strides=(4, 1), padding="valid", activation='relu')(x)
x = Convolution2D(257, kernel_size=(8, 1), strides=(4, 1), padding="valid", activation='relu')(x)
x = Reshape((173, 257, 1))(x)
x = Convolution2D(32, kernel_size=(3, 3), strides=(2, 2), padding="same", activation='relu')(x)
x = Convolution2D(71, kernel_size=(3, 3), strides=(2, 2), padding="same", activation='relu')(x)
x = Convolution2D(128, kernel_size=(4, 3), strides=(3, 2), padding="same", activation='relu')(x)
x = Convolution2D(128, kernel_size=(3, 3), strides=(2, 2), padding="same", activation='relu')(x)
x = Convolution2D(128, kernel_size=(3, 3), strides=(2, 2), padding="same", activation='relu')(x)
x = Convolution2D(128, kernel_size=(3, 3), strides=(2, 1), padding="same", activation='relu')(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(368, activation='sigmoid')(x)
# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=x)
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [541]:
print(X_train_samp.shape)
model.fit(X_train, Y_train, 
          batch_size=16, epochs=10, verbose=1, validation_split=0.1)

(5000, 16384, 1, 1)
Train on 4500 samples, validate on 500 samples
Epoch 1/10
4500/4500 [==============================] - 515s 114ms/sample - loss: 0.2356 - accuracy: 0.9375 - val_loss: 0.2355 - val_accuracy: 0.9375
Epoch 2/10
4500/4500 [==============================] - 468s 104ms/sample - loss: 0.2352 - accuracy: 0.9375 - val_loss: 0.2352 - val_accuracy: 0.9375
Epoch 3/10
4500/4500 [==============================] - 513s 114ms/sample - loss: 0.2350 - accuracy: 0.9375 - val_loss: 0.2351 - val_accuracy: 0.9375
Epoch 4/10
4500/4500 [==============================] - 540s 120ms/sample - loss: 0.2349 - accuracy: 0.9375 - val_loss: 0.2350 - val_accuracy: 0.9375
Epoch 5/10
4500/4500 [==============================] - 501s 111ms/sample - loss: 0.2348 - accuracy: 0.9375 - val_loss: 0.2348 - val_accuracy: 0.9375
Epoch 6/10
4500/4500 [==============================] - 537s 119ms/sample - loss: 0.2347 - accuracy: 0.9375 - val_loss: 0.2347 - val_accuracy: 0.9375
Epoch 7/10
4500/4500 [===========

In [472]:
model.build(input_shape=(45056, 1, 1))
print(model.summary())
print()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 16384, 1, 1)]     0         
_________________________________________________________________
batch_normalization_v2_93 (B (None, 16384, 1, 1)       4         
_________________________________________________________________
conv2d_463 (Conv2D)          (None, 4081, 1, 96)       6240      
_________________________________________________________________
conv2d_464 (Conv2D)          (None, 1013, 1, 96)       295008    
_________________________________________________________________
conv2d_465 (Conv2D)          (None, 250, 1, 128)       196736    
_________________________________________________________________
conv2d_466 (Conv2D)          (None, 61, 1, 257)        263425    
_________________________________________________________________
reshape_54 (Reshape)         (None, 61, 257, 1)        0   

In [542]:
o = model.predict(X_train)

In [540]:
print(o[400])
print(np.shape(o))

[0.05628255 0.04270926 0.06500456 0.06868699 0.07573196 0.066273
 0.07080418 0.0833357  0.05710584 0.07990831 0.06364202 0.09163126
 0.07093942 0.0479199  0.03154904 0.03957188 0.04017746 0.05546996
 0.08824024 0.06345186 0.05649811 0.05088392 0.08153579 0.0561156
 0.07138675 0.08023167 0.04997683 0.06974939 0.0493843  0.08742726
 0.07647741 0.05067453 0.04908192 0.08166295 0.0789291  0.06143141
 0.05200312 0.06106475 0.06709459 0.0614354  0.04765892 0.07334092
 0.07769474 0.05355269 0.05936807 0.06158039 0.06175458 0.06060976
 0.07592696 0.07929996 0.05385092 0.0678978  0.06181434 0.064372
 0.08036298 0.08136094 0.04289806 0.04069409 0.0859485  0.05049807
 0.05608943 0.08104172 0.07498387 0.05465728 0.06286219 0.07147077
 0.06457472 0.05262512 0.06081483 0.06714037 0.0990923  0.05345061
 0.06031105 0.06325069 0.05964991 0.06457436 0.06005648 0.03371531
 0.0949761  0.07652503 0.08784395 0.0524261  0.07132396 0.0593074
 0.06656966 0.06595549 0.08101723 0.09571379 0.04291567 0.05576515
 

In [536]:
group = 0
classes = np.zeros((23,1))
samp = 40
for i in range(0, 23):
    maxVal = o[samp, i * 16]
    maxI = 0
    for j in range(0, 16):
        v = o[samp, (i*16) + j]
        if v > maxVal:
            maxVal = o[samp, (i*16) + j]
            maxI = j
    classes[i] = maxI
print(classes)

[[11.]
 [ 2.]
 [ 1.]
 [10.]
 [ 6.]
 [ 7.]
 [13.]
 [12.]
 [ 3.]
 [ 4.]
 [ 0.]
 [11.]
 [ 5.]
 [ 2.]
 [ 2.]
 [ 6.]
 [ 8.]
 [ 9.]
 [ 6.]
 [ 8.]
 [10.]
 [14.]
 [ 9.]]
